# LSTM for sequence classification in the IMDB dataset

#### Keras provided imdb dataset in the following manner:
#### first construct set of all the words in all the reviews.
#### construct a table with columns-- word, frequency.
#### sort this table based on the frequency of words
#### Then given them as numbers like first as 1 and then 2 , continue.
#### DATASET LINK--https://keras.io/api/datasets/imdb/ 

In [ ]:
# Importing dependencies
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
tf.random.set_seed(7)

In [ ]:
# load the dataset but only keep the top n words, zero the rest
# 25k in train and 25k in test
top_words = 6000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
print(X_train[0])
print(type(X_train[0]))
print(len(X_train[0]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
<class 'list'>
218


##### second review contains 218 words.

In [ ]:

# truncate and pad the input sequences, so they are all the same length for modeling. The model will learn that the zero values carry 
#no information. The sequences are not the same length in terms of content, but same-length vectors are required to perform 
#the computation in Keras.
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[0])

(25000, 600)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [ ]:
#create the model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(105))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 600, 32)           192000    
                                                                 
 lstm_6 (LSTM)               (None, 105)               57960     
                                                                 
 dense_6 (Dense)             (None, 1)                 106       
                                                                 
Total params: 250,066
Trainable params: 250,066
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
checkpoint_filepath = '/content/gdrive/MyDrive/LSTM_model_checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model.fit(X_train, y_train, epochs=12, batch_size=64,callbacks=[model_checkpoint_callback])
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
model.save('/content/gdrive/MyDrive/LSTM_model_checkpoints/')
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/12
391/391 [==============================] - 14s 32ms/step - loss: 0.4345 - accuracy: 0.7966
Epoch 2/12
391/391 [==============================] - 12s 31ms/step - loss: 0.2853 - accuracy: 0.8854
Epoch 3/12
391/391 [==============================] - 12s 32ms/step - loss: 0.2372 - accuracy: 0.9087
Epoch 4/12
391/391 [==============================] - 12s 32ms/step - loss: 0.1967 - accuracy: 0.9259
Epoch 5/12
391/391 [==============================] - 12s 32ms/step - loss: 0.1777 - accuracy: 0.9337
Epoch 6/12
391/391 [==============================] - 13s 33ms/step - loss: 0.1948 - accuracy: 0.9254
Epoch 7/12
391/391 [==============================] - 13s 33ms/step - loss: 0.1520 - accuracy: 0.9438
Epoch 8/12
391/391 [==============================] - 13s 33ms/step - loss: 0.1314 - accuracy: 0.9524
Epoch 9/12
391/391 [==============================] - 13s 33ms/step - loss: 0.1495 - accuracy: 0.9458
Epoch 10/12
391/391 [==============================] - 13s 33ms/step - loss: 0.111

Accuracy: 85.55%


### Using Dropout Layer

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=12, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 600, 32)           192000    
                                                                 
 dropout_2 (Dropout)         (None, 600, 32)           0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               53200     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 245,301
Trainable params: 245,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/12
391/391 [=======================